# Pryngles module: Science

In [36]:
from pryngles import *

## External modules

In [37]:
import numpy as np
import math as mh
import spiceypy as spy

verbose=Verbose.print

## The Science class

The Science class is a class with routines intended to perform a wide diversity of mathematical, physical and astronomical calculations.

In [38]:
class Science(PrynglesCommon):pass

### Template method

In [39]:
def template(foo=1):
    """
    Method

    Parameters:

        foo: type [units], default = 1:
            Description.

    Return:

        fee: type [units]:
            Desctiption.

    """
    return foo

Science.template=template

In [40]:
if IN_JUPYTER:
    def test_temp(self):
        
        #Test
        Science.template()
        """
        self.assertEqual(self.P.Nr,8,True)
        self.assertEqual(np.isclose([P.physics.wrot],
                                    [2*np.pi/PlanetDefaults.physics["prot"]],
                                    rtol=1e-7),
                         [True]*1)
        self.assertRaises(AssertionError,lambda:Observer(primary="Nada"))
        """
        
    class Test(unittest.TestCase):pass
    Test.test_temp=test_temp
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Cartesian to spherical

In [41]:
def spherical(xyz):
    """
    Transform cartesian coordinates into spherical coordinates

    Parameters:

        xyz: array (3):
            Cartesian coordinates

    Return:

        rqf: array (3):
            Spherical coordinates (r, theta, phi) where theta is azimutal angle and phi is 
            elevation (complement of polar angle).                

            Notice that this convention is different than that of regular vectorial calculus
            where spherical coordinates are (r,theta,phi), but theta is the polar angle and phi 
            the ezimutal one.

    """
    r,theta,phi=spy.reclat(np.array(xyz))
    theta=2*mh.pi+theta if theta<0 else theta

    return np.array([r,theta,phi])

def cartesian(rqf):
    """
    Transform cartesian coordinates into spherical coordinates

    Parameters:

        xyz: array (3):
            Cartesian coordinates

    Return:

        rqf: array (3):
            Spherical coordinates (r, theta, phi) where theta is azimutal angle and phi is 
            elevation (complement of polar angle).                

            Notice that this convention is different than that of regular vectorial calculus
            where spherical coordinates are (r,theta,phi), but theta is the polar angle and phi 
            the ezimutal one.

    """
    return spy.latrec(rqf[0],rqf[1],rqf[2])

Science.spherical=spherical
Science.cartesian=cartesian

In [42]:
if IN_JUPYTER:
    def test_coords(self):
        
        #Test spherical
        rqf=Science.spherical([1,1,0])
        print(rqf[0],rqf[1]*Consts.rad,rqf[2]*Consts.rad)   
        rqf=Science.spherical([+1,+1,1])
        print(rqf[0],rqf[1]*Consts.rad,rqf[2]*Consts.rad)   
        rqf=Science.spherical([-1,1,1])
        print(rqf[0],rqf[1]*Consts.rad,rqf[2]*Consts.rad)   
        rqf=Science.spherical([-1,-1,1])
        print(rqf[0],rqf[1]*Consts.rad,rqf[2]*Consts.rad)   
        rqf=Science.spherical([+1,-1,1])
        print(rqf[0],rqf[1]*Consts.rad,rqf[2]*Consts.rad)   
        rqf=Science.spherical([+1,+1,-1])
        print(rqf[0],rqf[1]*Consts.rad,rqf[2]*Consts.rad)   
        rqf=Science.spherical([-1,1,-1])
        print(rqf[0],rqf[1]*Consts.rad,rqf[2]*Consts.rad)   
        rqf=Science.spherical([-1,-1,-1])
        print(rqf[0],rqf[1]*Consts.rad,rqf[2]*Consts.rad)   
        rqf=Science.spherical([+1,-1,-1])
        print(rqf[0],rqf[1]*Consts.rad,rqf[2]*Consts.rad)
        
        #Test cartesian
        xyz=Science.cartesian([1,0,0])
        print(xyz) 
        xyz=Science.cartesian([1,45*Consts.deg,45*Consts.deg])
        print(xyz) 
        xyz=Science.cartesian([1,135*Consts.deg,45*Consts.deg])
        print(xyz) 
        xyz=Science.cartesian([1,225*Consts.deg,45*Consts.deg])
        print(xyz) 
        xyz=Science.cartesian([1,315*Consts.deg,45*Consts.deg])
        print(xyz) 
        xyz=Science.cartesian([1,45*Consts.deg,-45*Consts.deg])
        print(xyz) 
        xyz=Science.cartesian([1,135*Consts.deg,-45*Consts.deg])
        print(xyz) 
        xyz=Science.cartesian([1,225*Consts.deg,-45*Consts.deg])
        print(xyz) 
        xyz=Science.cartesian([1,315*Consts.deg,-45*Consts.deg])
        print(xyz) 

    class Test(unittest.TestCase):pass
    Test.test_coords=test_coords
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

.

1.4142135623730951 45.0 0.0
1.7320508075688772 45.0 35.264389682754654
1.7320508075688772 135.0 35.264389682754654
1.7320508075688772 225.0 35.264389682754654
1.7320508075688772 315.0 35.264389682754654
1.7320508075688772 45.0 -35.264389682754654
1.7320508075688772 135.0 -35.264389682754654
1.7320508075688772 225.0 -35.264389682754654
1.7320508075688772 315.0 -35.264389682754654
[1. 0. 0.]
[0.5        0.5        0.70710678]
[-0.5         0.5         0.70710678]
[-0.5        -0.5         0.70710678]
[ 0.5        -0.5         0.70710678]
[ 0.5         0.5        -0.70710678]
[-0.5         0.5        -0.70710678]
[-0.5        -0.5        -0.70710678]
[ 0.5        -0.5        -0.70710678]



----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


In [45]:
def rotation_matrix(ez,alpha):
    """
    Set a rotation matrix from the direction of the ez vector and a rotation angle alpha
    
    Parameter:
        ez: array (3)
            vector in the direction of the z-axis. 
            
        alpha: float (3) [rad]
            Rotation angle of the x-axis around z-axis (clockwise)
            
    Return:
        Msys2uni: float (3x3)
            Rotation matrix from the system defined by ez and the universal system.
            
        Muni2sys: float (3x3)
            Rotation matrix from the universal system to the system defined by ez
    """
    ez,one=spy.unorm(ez)
    ex=spy.ucrss([0,0,1],ez) #Spice is 5 faster for vcrss
    if spy.vnorm(ex)==0:
        ex=np.array([1,0,0]) if np.sum(ez)>0 else np.array([-1,0,0])
    ey=spy.ucrss(ez,ex)
    Msys2uni=np.array(list(np.vstack((ex,ey,ez)).transpose())).reshape((3,3))
    Muni2sys=spy.invert(Msys2uni)
    verbose("Rotation axis:",ex,ey,ez)
    return Msys2uni,Muni2sys

Science.rotation_matrix=rotation_matrix

In [44]:
if IN_JUPYTER:
    def test_rot(self):
        Verbose.VERBOSITY=1
        
        #Test rotation
        Msys2uni,Muni2sys=Science.rotation_matrix([0,0,1],0)
        print(Msys2uni)

        Msys2uni,Muni2sys=Science.rotation_matrix([0,0,-1],0)
        print(Msys2uni)

        Verbose.VERBOSITY=0

    class Test(unittest.TestCase):pass
    Test.test_rot=test_rot
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

.

Rotation axis: [1 0 0] [0. 1. 0.] [0. 0. 1.]
[[ 1. -0.  0.]
 [-0.  1. -0.]
 [ 0. -0.  1.]]
Rotation axis: [-1  0  0] [0. 1. 0.] [ 0.  0. -1.]
[[-1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]



----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


--End--